In [2]:
import numpy as np
import pandas as pd 
import traceback
import yfinance as yf

# Helper class: Fetch Stock Data 

To Do List: 
* Get data within a specific range

In [3]:
class StockFetcher: 
    def __fetch_historical_data(self, ticker, time_unit, period=None): 
        try:
            if not period: 
                if time_unit in ['ytd', 'max']: 
                    period_str = f'{time_unit}'
                else:
                    raise ValueError('Period must be defined for a given time unit')
            else: 
                period_str = f'{period}{time_unit}'

            data = yf.Ticker(ticker).history(period=period_str)
            return data 
        except Exception as e: 
            print(f'Validation error: {e}')
            traceback.print_exc()
            return None 

    def get_stock_data(self, ticker, time_unit, period=None): 
        if time_unit not in ['d', 'mo', 'y', 'ytd', 'max']: 
            raise ValueError('Time unit not valid') # Later be changed to a HTTP error 
        
        match time_unit: 
            case 'd': 
                if period not in [1, 5]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'mo': 
                if period not in [1, 3, 6]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'y': 
                if period not in [1, 2, 5, 10]: 
                    raise ValueError('Period not valid') # Later be changed to a HTTP error 
            case 'ytd' | 'max': 
                period = None
            case _: 
                raise ValueError('Time unit not valid')
        
        try: 
            data = self.__fetch_historical_data(ticker, time_unit, period)
            return data
        except Exception as e: 
            print(f'Validation error: {e}')
            traceback.print_exc()
            return None

# Helper Class: Performs Regression Analysis 

To Do List: 
* Regression analysis within a specific range 

In [7]:
class LinearRegresion: 

    def __get_coefficients(self, asset, market):
        avg_market_return = market['Returns'].mean()
        avg_asset_return = asset['Returns'].mean()

        covariance = market['Returns'].cov(asset['Returns'])
        variance = market['Returns'].var() 

        beta = covariance / variance 
        alpha = avg_asset_return - beta * avg_market_return

        expected_returns = alpha + beta*avg_market_return
        
        return alpha, beta, expected_returns

    def linear_regression(self, ticker): 
        stock_fetcher = StockFetcher() 
        asset = stock_fetcher.get_stock_data(ticker=ticker, time_unit='y', period=10)    
        market = stock_fetcher.get_stock_data(ticker='SPY', time_unit='y', period=10)    

        date_match = pd.merge(asset, market, left_index=True, right_index=True, how='inner').index 

        asset_filtered = asset.loc[date_match]
        market_filtered = market.loc[date_match]

        asset_filtered['Returns'] = asset_filtered.Close.pct_change()
        market_filtered['Returns'] = market_filtered.Close.pct_change()

        alpha, beta, expected_returns = self.__get_coefficients(asset_filtered, market_filtered)

        return alpha, beta, expected_returns

lg = LinearRegresion() 

for ticker in ['AAPL', 'NVDA', 'GOOG']: 
    alpha, beta, expected_returns = LinearRegresion().linear_regression(ticker)
    print(f'Ticker: {ticker}\n\t- Excess Returns: {alpha}\n\t- Market Relation: {beta}\n\t- Expected Returns: {expected_returns}%')

Ticker: AAPL
	- Excess Returns: 0.0004157690459561409
	- Market Relation: 1.2098946658816967
	- Expected Returns: 0.0010865688364017536%
Ticker: NVDA
	- Excess Returns: 0.0017250249298859479
	- Market Relation: 1.7478951317216191
	- Expected Returns: 0.0026941073807332967%
Ticker: GOOG
	- Excess Returns: 0.0002145187965467393
	- Market Relation: 1.1555181452128969
	- Expected Returns: 0.0008551707074302819%
